# Lab 11: Grid Localization using Bayes Filter (Virtual Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

<hr>


In [1]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
from localization_extras import Localization

# Setup Logger
LOG = get_logger('demo_notebook.log')

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

gui.show()

# Start the simulator
START_SIM()

# Start the plotter
START_PLOTTER()

2025-04-27 15:08:28,565 | INFO     |: Logger demo_notebook.log initialized.


TwoByTwoLayout(children=(Label(value='Simulator', layout=Layout(grid_area='top-left', width='80px')), HBox(chi…

In [2]:
# Initialize Robot to communicate with the virtual robot and plotter
robot = VirtualRobot(cmdr)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a VirtualRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2025-04-27 15:08:35,715 | INFO     |:  | Number of observations per grid cell: 18
2025-04-27 15:08:35,717 | INFO     |:  | Precaching Views...


/Users/sana/Desktop/FastRobots/FastRobots-sim-release-main/localization.py:150: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(distance_intersections_tt), intersections_tt[np.nanargmin(distance_intersections_tt)]


2025-04-27 15:08:36,772 | INFO     |:  | Precaching Time: 1.055 secs
2025-04-27 15:08:36,773 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-04-27 15:08:36,773 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


# Run the Bayes Filter
The cells below utilizes the member functions of class **Localization** (defined in [localization_extras.py](../localization_extras.py)) in each iteration of the Bayes filter algorithm to localize the robot in the grid map. <br>

In [3]:
# Reset Robot and Plots
robot.reset()
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.print_update_stats(plot_data=True)

# Plot Odom and GT
current_odom, current_gt = robot.get_pose()
cmdr.plot_gt(current_gt[0], current_gt[1])
cmdr.plot_odom(current_odom[0], current_odom[1])

2025-04-27 15:08:59,300 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-04-27 15:08:59,301 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
2025-04-27 15:09:02,219 | INFO     |: Update Step
2025-04-27 15:09:02,220 | INFO     |:      | Update Time: 0.001 secs
2025-04-27 15:09:02,220 | INFO     |: ---------- UPDATE STATS -----------
2025-04-27 15:09:02,229 | INFO     |: GT index      : (6, 4, 9)
2025-04-27 15:09:02,229 | INFO     |: Bel index     : (np.int64(5), np.int64(4), np.int64(9)) with prob = 0.9998531
2025-04-27 15:09:02,229 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2025-04-27 15:09:02,230 | INFO     |: GT            : (0.000, 0.000, 360.000)
2025-04-27 15:09:02,231 | INFO     |: Belief        : (0.000, 0.000, 10.000)
2025-04-27 15:09:02,231 | INFO     |: POS ERROR     : (-0.000, -0.000, 350.000)
2025-04-27 15:09:02,232 | INFO     |: ---------- UPDATE STATS -----------


In [4]:
# Initialize the Trajectory object
traj = Trajectory(loc)

# Run through each motion steps
for t in range(0, traj.total_time_steps):
    print("\n\n-----------------", t, "-----------------")
    
    prev_odom, current_odom, prev_gt, current_gt = traj.execute_time_step(t)
        
    # Prediction Step
    loc.prediction_step(current_odom, prev_odom)
    loc.print_prediction_stats(plot_data=True)
    
    # Get Observation Data by executing a 360 degree rotation motion
    loc.get_observation_data()
    
    # Update Step
    loc.update_step()
    loc.print_update_stats(plot_data=True)

# Uncomment the below line to wait for keyboard input between each iteration.
#   input("Press Enter to Continue")
        
    print("-------------------------------------")



----------------- 0 -----------------
2025-04-27 15:09:07,510 | INFO     |: Prediction Step
2025-04-27 15:09:07,540 | INFO     |:  | Prediction Time: 0.029 secs
2025-04-27 15:09:07,540 | INFO     |: ---------- PREDICTION STATS -----------
2025-04-27 15:09:07,543 | INFO     |: GT index         : (6, 3, 7)
2025-04-27 15:09:07,544 | INFO     |: Prior Bel index  : (np.int64(8), np.int64(3), np.int64(8)) with prob = 0.0736550
2025-04-27 15:09:07,544 | INFO     |: POS ERROR        : (-0.628, 0.216, -29.725)
2025-04-27 15:09:07,545 | INFO     |: ---------- PREDICTION STATS -----------
2025-04-27 15:09:10,529 | INFO     |: Update Step
2025-04-27 15:09:10,530 | INFO     |:      | Update Time: 0.001 secs
2025-04-27 15:09:10,530 | INFO     |: ---------- UPDATE STATS -----------
2025-04-27 15:09:10,546 | INFO     |: GT index      : (6, 3, 7)
2025-04-27 15:09:10,546 | INFO     |: Bel index     : (np.int64(6), np.int64(4), np.int64(6)) with prob = 1.0
2025-04-27 15:09:10,547 | INFO     |: Bel_bar 

Traceback (most recent call last):
  File "/Users/sana/Desktop/FastRobots/FastRobots-sim-release-main/src/plotter.py", line 263, in keyPressEvent
    if event.key() == Qt.Key_Escape:
                      ^^^^^^^^^^^^^
AttributeError: type object 'Qt' has no attribute 'Key_Escape'
Traceback (most recent call last):
  File "/Users/sana/Desktop/FastRobots/FastRobots-sim-release-main/src/plotter.py", line 263, in keyPressEvent
    if event.key() == Qt.Key_Escape:
                      ^^^^^^^^^^^^^
AttributeError: type object 'Qt' has no attribute 'Key_Escape'
Traceback (most recent call last):
  File "/Users/sana/Desktop/FastRobots/FastRobots-sim-release-main/src/plotter.py", line 252, in closeEvent
    result = QtGui.QMessageBox.question(self,
             ^^^^^^^^^^^^^^^^^
AttributeError: module 'pyqtgraph.Qt.QtGui' has no attribute 'QMessageBox'


Loading Flatland...
Initializing pygame framework...
